### Import rdflib and create functions

* g is the ontology graph where the infered relations are added to
* Serialize is a function that transforms the ontology into a turle format
* Save is a function that can save the ontology as a turtle file
* Load is a function that loads the ontology into this file
* Shorten is a function that shows the prefix + subject, property or object vb. kinship1:has_mother
* Relation shows only the subject, property or object

In [1]:
import rdflib
from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef 
import pandas as pd

pd.set_option('display.max_rows', 999)


df = pd.DataFrame()
g = Graph()

KIN = Namespace('http://www.semanticweb.org/combots#')
g.bind('kinship1',KIN)

def serialize():
    print(g.serialize(format='turtle').decode("utf-8"))

def save(filename):
    with open(filename, 'wb') as f:
        g.serialize(f, format='turtle')
        
def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle') 
    
def shorten(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s:%s'%(components[0], components[2])

def relation(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s'%(components[2]) 

## Create counter that counts the amount of newly created relations in this ontology.

In [2]:
new_relations_counter = 0

### Load in the ontology and print in turtle format

In [3]:
load('raw_output_ontology.ttl')
serialize()

@prefix kinship1: <http://www.semanticweb.org/combots#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

kinship1:Big_Nosed_Rachel kinship1:has_Parents kinship1:Fat_Monica .

kinship1:Danny kinship1:has_Sister kinship1:Rachel .

kinship1:Doug kinship1:has_Mother kinship1:Chandler .

kinship1:Dr._Green kinship1:has_Dauter kinship1:Ross .

kinship1:Elizabeth kinship1:has_Dad kinship1:Rachel ;
    kinship1:has_Father kinship1:Ross .

kinship1:Joanne kinship1:has_Dad kinship1:Kiki .

kinship1:Joshua kinship1:has_Parents kinship1:Rachel .

kinship1:Mrs._Bing kinship1:has_Son kinship1:All,
        kinship1:Chandler .

kinship1:Mrs._Tribbiani kinship1:has_Father kinship1:Joey .

<http://www.semanticweb.org/combots#Mrs._Verhoeven's_Daughter> kinship1:has_Mother kinship1:Ross .

kinship1:Pizza_Guy kinship1:has_Dad kinship1:Monica .

kinship1:ROSS kinship1:has_Sister kinship1:MNCA .

kinship1:Tag kinship1:has_Sister kinship1:Rachel .

kinship1:Tom kinship1:has_Mom kinship1:Rachel .

kinship1:D

### Create empty lists that have to be filled later.

In [4]:
parent  = []
parent2 = []
father  = []
mother  = []
child   = []
child2  = []
son     = []
daughter = []
grandchild = []
grandchild2 = []
grandparent = []
grandparent2 = []
couple  = []
couple2 = []
marriage = []
marriage2 = []
marriage3 = []
marriage4 = []
marriage5 = []
marriage6 = []
marriage7 = []
marriage8 = []
cousin1 = []
cousin2 = []
sibling1 = []
sibling2 = []
sister = []
sister2 = []
brother = []
brother2 = []

### Create dataframes that later shows the inferred gender relations and  the inferred relations.

In [5]:
df = pd.DataFrame(columns = ["Relation", "Subject", "Property", "Object"])
df2 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
df5 = pd.DataFrame(columns = ["Subject", "Property", "Object"])

 ### Fill the lists with the right values and add gender to individuals
 The known relations will be added seperatly in lists and the genders of the individuals will be added to the ontology by determining what their relation is with other individuals. For example, if someone is a mother she will automatically also be a woman.

In [6]:
for s,p,o in g:
# PARENT RELATIONS
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        parent.append(o)
        child.append(s)    
    if shorten(p) == "kinship1:has_Father":
        df = df.append({'Relation': "Father", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Mother":
        df = df.append({'Relation': "Mother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# GRANDPARENT RELATIONS
    if shorten(p) == "kinship1:has_Grandparent" or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        grandparent.append(o)
        grandchild.append(s)    
    if shorten(p) == "kinship1:has_Grandfather":
        df = df.append({'Relation': "Grandfather", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Grandmother":
        df = df.append({'Relation': "Grandmother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# CHILD RELATIONS       
    if shorten(p) == "kinship1:has_Child" or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther": #typo in ontology daughter
        child2.append(o)
        parent2.append(s)    
    if shorten(p) == "kinship1:has_Son":
        df = df.append({'Relation': "Son", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Dauther":
        df = df.append({'Relation': "Daughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# GRANDCHILD RELATIONS       
    if shorten(p) == "kinship1:has_Grandchild" or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter": 
        grandchild2.append(o)
        grandparent2.append(s)   
    if shorten(p) == "kinship1:has_Grandson":
        df = df.append({'Relation': "Grandson", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Granddaughter":
        df = df.append({'Relation': "Granddaughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# SIBLING RELATIONS
    if shorten(p) == "kinship1:has_Sibling" or  shorten(p) == "kinship1:has_Sister" or shorten(p) == "kinship1:has_Brother": 
        sibling1.append(o)
        sibling2.append(s)
    if shorten(p) == "kinship1:has_Brother":
        brother.append(o)
        brother2.append(s)
        df = df.append({'Relation': "Brother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Sister":
        sister.append(o)
        sister2.append(s)
        df = df.append({'Relation': "Sister", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# COUSIN RELATIONS
    if shorten(p) == "kinship1:has_Cousin":
        cousin1.append(o)
        cousin2.append(s)

#RELATIONSHIPS 
    if shorten(p) == "kinship1:has_Husband" or shorten(p) == "kinship1:has_Wife" or shorten(p) == "kinship1:has_Partner":
        couple.append(o)
        couple2.append(s)
    if shorten(p) == "kinship1:has_Husband":
        marriage.append(o)
        marriage2.append(s)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
        df = df.append({'Relation': "Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Wife":
        marriage3.append(o)
        marriage4.append(s)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_husband":
        marriage5.append(o)
        marriage6.append(s)
        g.add((o, RDFS.domain, KIN.man)) 
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_wife":
        marriage7.append(o)
        marriage8.append(s)
        g.add((o, RDFS.domain, KIN.woman))    
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)

### Print a table with the inferred gender relations that are added to the ontology

In [7]:
df = df.drop(columns=["Property"])
df = df.sort_values(by=['Relation'])
df.reset_index(drop=True)

,Relation,Subject,Object
0,Brother,Paul,man
1,Brother,Joey,man
2,Brother,Nurse,man
3,Brother,Joey,man
4,Brother,Joey,man
5,Brother,Monica,man
6,Brother,Angela,man
7,Brother,Rachel,man
8,Brother,Joey,man
9,Brother,Ross,man


In [8]:
print (g.serialize(format='turtle').decode("utf-8")) 

@prefix kinship1: <http://www.semanticweb.org/combots#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

kinship1:Big_Nosed_Rachel kinship1:has_Parents kinship1:Fat_Monica .

kinship1:Danny kinship1:has_Sister kinship1:Rachel .

kinship1:Doug kinship1:has_Mother kinship1:Chandler .

kinship1:Dr._Green kinship1:has_Dauter kinship1:Ross .

kinship1:Elizabeth kinship1:has_Dad kinship1:Rachel ;
    kinship1:has_Father kinship1:Ross .

kinship1:Joanne kinship1:has_Dad kinship1:Kiki .

kinship1:Joshua kinship1:has_Parents kinship1:Rachel .

kinship1:Mrs._Bing kinship1:has_Son kinship1:All,
        kinship1:Chandler .

kinship1:Mrs._Tribbiani kinship1:has_Father kinship1:Joey .

<http://www.semanticweb.org/combots#Mrs._Verhoeven's_Daughter> kinship1:has_Mother kinship1:Ross .

kinship1:Pizza_Guy kinship1:has_Dad kinship1:Monica .

kinship1:ROSS kinship1:has_Sister kinship1:MNCA .

kinship1:Tag kinship1:has_Sister kinship1:Rachel .

kin

### Extract all the necessary properties and print the ones that exist in the ontology to show what they look like.
The ontology might consist of many more relations,but these are similar to these ones. They will however not be detected if they are not equal to the default properties. That would have been a lot more work to find all the relations and due to limited time, only the default properties can be used. 

In [9]:
child_property = None
parent_property = None 
grandchild_property= None
grandparent_property= None
husband_property= None
wife_property= None
ex_husband_property= None
ex_wife_property= None
cousin_property= None
partner_property= None
father_property= None
mother_property= None
son_property= None
daughter_property= None
grandson_property= None
granddaugther_property= None
grandfather_property= None
grandmother_property= None
brother_property= None
sister_property= None
twin_property= None
sibling_property= None

In [10]:
propery_list = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_Child":
        child_property = p
        propery_list.append(p)
        print("child_property", child_property)
    if shorten(p) == "kinship1:has_parent":
        parent_property = p
        propery_list.append(p)
        print("parent_property", parent_property)
    if shorten(p) == "kinship1:has_Grandchild":
        grandchild_property = p
        propery_list.append(p)
        print("grandchild_property", grandchild_property)
    if shorten(p) == "kinship1:has_Grandparent":
        grandparent_property = p
        propery_list.append(p)
        print("grandparent_property", grandparent_property)
        
    if shorten(p) == "kinship1:has_Husband":
        husband_property = p
        propery_list.append(p)
        print("husband_property", husband_property)
    if shorten(p) == "kinship1:has_Wife":
        wife_property = p
        propery_list.append(p)
        print("wife_property", wife_property)
    if shorten(p) == "kinship1:has_Ex_husband":
        ex_husband_property = p
        propery_list.append(p)
        print("ex_husband_property", ex_husband_property)
    if shorten(p) == "kinship1:has_Ex_wife":
        ex_wife_property = p
        propery_list.append(p)
        print("ex_wife_property", ex_wife_property)
            
    if shorten(p) ==  "kinship1:has_Cousin":
        cousin_property = p
        propery_list.append(p)
        print("cousin_property", cousin_property)
    if shorten(p) == "kinship1:has_Sibling":
        sibling_property = p
        propery_list.append(p)
        print("sibling_property", sibling_property)
            

    if shorten(p) == "kinship1:has_Partner":
        partner_property = p
        propery_list.append(p)
        print("partner_property", partner_property)
    if shorten(p) == "kinship1:has_Father":
        father_property = p
        propery_list.append(p)
        print("father_property", father_property)
    if shorten(p) == "kinship1:has_Mother":
        mother_property = p
        propery_list.append(p)
        print("mother_property", mother_property)
    if shorten(p) ==  "kinship1:has_Son"  :
        son_property = p
        propery_list.append(p)
        print("son_property", son_property)
    if shorten(p) == "kinship1:has_Dauther":
        daugther_property = p
        propery_list.append(p)
        print("daugther_property", daugther_property)
    if shorten(p) == "kinship1:has_Grandson":
        grandson_property = p
        propery_list.append(p)
        print("grandson_property", grandson_property)
    if shorten(p) == "kinship1:has_Granddaughter":
        granddaugther_property = p
        propery_list.append(p)
        print("granddaugther_property", granddaugther_property)
    if shorten(p) == "kinship1:has_Grandfather":
        grandfather_property = p
        propery_list.append(p)
        print("grandfather_property", grandfather_property)
    if shorten(p) == "kinship1:has_Grandmother":
        grandmother_property = p
        propery_list.append(p)
        print("grandmother_property", grandmother_property)
    if shorten(p) == "kinship1:has_Brother":
        brother_property = p
        propery_list.append(p)
        print("brother_property", brother_property)
    if shorten(p) == "kinship1:has_Sister":
        sister_property = p
        propery_list.append(p)
        print("sister_property", sister_property)
    if shorten(p) == "kinship1:has_twin":
        twin_property = p
        propery_list.append(p)
        print("twin_property", twin_property)


son_property http://www.semanticweb.org/combots#has_Son
mother_property http://www.semanticweb.org/combots#has_Mother
sister_property http://www.semanticweb.org/combots#has_Sister
son_property http://www.semanticweb.org/combots#has_Son
mother_property http://www.semanticweb.org/combots#has_Mother
brother_property http://www.semanticweb.org/combots#has_Brother
sister_property http://www.semanticweb.org/combots#has_Sister
brother_property http://www.semanticweb.org/combots#has_Brother
mother_property http://www.semanticweb.org/combots#has_Mother
mother_property http://www.semanticweb.org/combots#has_Mother
sister_property http://www.semanticweb.org/combots#has_Sister
twin_property http://www.semanticweb.org/combots#has_twin
father_property http://www.semanticweb.org/combots#has_Father
child_property http://www.semanticweb.org/combots#has_Child
sister_property http://www.semanticweb.org/combots#has_Sister
son_property http://www.semanticweb.org/combots#has_Son
son_property http://www.sema

### Divine some necessary properties
These property names are used as a default for these relations. And since these are very important kinship relations, they are added if they are missing in the input data.

In [11]:
if sibling_property == None:
    sibling_property =  rdflib.term.URIRef(KIN.has_Sibling)
SymmetricProperty = rdflib.term.URIRef(KIN.SymmetricProperty)
SymmetricProperty = URIRef(SymmetricProperty)
sibling_property=URIRef(sibling_property)
print(sibling_property, RDF.type, SymmetricProperty)
g.add((sibling_property, RDF.type, SymmetricProperty))

if parent_property == None:
    parent_property =   rdflib.term.URIRef(KIN.has_parent)
IrreflexiveProperty = rdflib.term.URIRef(KIN.IrreflexiveProperty)
IrreflexiveProperty = URIRef(IrreflexiveProperty)
parent_property=URIRef(parent_property)
print(parent_property, RDF.type, IrreflexiveProperty)
g.add((parent_property, RDF.type, IrreflexiveProperty))

if child_property == None:
    child_property =   rdflib.term.URIRef(KIN.has_Child)
IrreflexiveProperty = rdflib.term.URIRef(KIN.IrreflexiveProperty)
IrreflexiveProperty = URIRef(IrreflexiveProperty)
child_property=URIRef(child_property)
print(child_property, RDF.type, IrreflexiveProperty)
g.add((child_property, RDF.type, IrreflexiveProperty))

if daughter_property == None:
    daughter_property =   rdflib.term.URIRef(KIN.has_daughter)
IrreflexiveProperty = rdflib.term.URIRef(KIN.IrreflexiveProperty)
IrreflexiveProperty = URIRef(IrreflexiveProperty)
daughter_property=URIRef(daughter_property)
print(daughter_property, RDF.type, IrreflexiveProperty)
g.add((daughter_property, RDF.type, IrreflexiveProperty))

http://www.semanticweb.org/combots#has_Sibling http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/combots#SymmetricProperty
http://www.semanticweb.org/combots#has_parent http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/combots#IrreflexiveProperty
http://www.semanticweb.org/combots#has_Child http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/combots#IrreflexiveProperty
http://www.semanticweb.org/combots#has_daughter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/combots#IrreflexiveProperty


In [12]:
property_list = list(dict.fromkeys(propery_list))
property_list

[rdflib.term.URIRef('http://www.semanticweb.org/combots#has_Son'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_Mother'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_Sister'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_Brother'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_twin'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_Father'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_Child'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_Cousin'),
 rdflib.term.URIRef('http://www.semanticweb.org/combots#has_parent')]

### Create functions to iterate through the lists and add the reversed relations with the right property to the ontology.
All the added relations are also added to a dataframe.

In [13]:
def create_inferrence(new_subject, new_property, new_object, ignore_index=True):
    global df2
    global new_relations_counter
    for i in range(len(new_subject)):
        g.add((new_subject[i], new_property, new_object[i]))
        new_relations_counter +=1
        data = [[ relation(new_subject[i]), relation(new_property), relation(new_object[i])]]
        df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
        df2 = df2.append(df3)
        
def create_gender_based_inferrence(subject_list, new_property, relationship, ignore_index=True):
    global df5        
    global new_relations_counter
    for x in subject_list:
        if x[1] in relationship:
            g.add((x[0], new_property, x[1]))
            new_relations_counter +=1
            data = [[ relation(x[0]), relation(new_property), relation(x[1])]]
            df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
            df5 = df5.append(df3)
            
def make_word_classes(synom):
    # List of classes
    word_classes = []
    for w1, w2 in synom:
        # Go through existing classes
        for wcls in word_classes:
            # If one of the words is already in the current class
            if w1 in wcls or w2 in wcls:
                # Add both words and continue to next pair of words
                wcls.add(w1)
                wcls.add(w2)
                break
        else:  # Note this else goes with the for loop, not the if block
            # If there was no matching class, add a new one
            word_classes.append({w1, w2})
    return word_classes

### Create relations that are based on an individuals gender.

In [14]:
woman_list = []
man_list = []
has_parent_relation = []
has_child_relation = []
has_grandparent_relation = []
has_grandchild_relation = []
has_sibling_relation = []
has_twin_relation=[]

for s,p,o in g:
    if shorten(o) ==  "kinship1:woman":
        woman_list.append(s)
    if shorten(o) ==  "kinship1:man":
        man_list.append(s)
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        has_parent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Child" or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther":
        has_child_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandparent" or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        has_grandparent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandchild" or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter":
        has_grandchild_relation.append((o, s))
    if shorten(p) == "kinship1:has_Sibling" or shorten(p) == "kinship1:has_Sister" or shorten(p) == "kinship1:has_Brother":
        has_sibling_relation.append((o,s))
    if shorten(p) == "kinship1:has_twin":
        has_twin_relation.append((o,s))

In [15]:
mothers = set(woman_list) & set([x[0] for x in has_parent_relation])
fathers = set(man_list) & set([x[0] for x in has_parent_relation])
daughters = set(woman_list) & set([x[0] for x in has_child_relation])
sons = set(man_list) & set([x[0] for x in has_child_relation])

daughters2 =  set(woman_list) & set([x[1] for x in has_parent_relation])
sons2 = set(man_list) & set([x[1] for x in has_parent_relation])
sons.update(sons2)
daughters.update(daughters2)                                    

brothers = set(man_list) & set([x[0] for x in has_sibling_relation])
twin_sisters = set(woman_list) & set([x[0] for x in has_twin_relation])
twin_brothers = set(man_list) & set([x[0] for x in has_twin_relation])
sisters = set(woman_list) & set(x[1] for x in has_sibling_relation)

grandmothers = set(woman_list) & set([x[0] for x in has_grandparent_relation])
grandfathers = set(man_list) & set([x[0] for x in has_grandparent_relation])
granddaughters = set(woman_list) & set([x[0] for x in has_grandchild_relation])
grandsons = set(man_list) & set([x[0] for x in has_grandchild_relation])

# mothers, fathers, daughters, sons, grandmothers, grandfathers, granddaughters, grandsons, sisters, brothers,twin_sisters,twin_brothers

In [16]:
def inferences():
    if child_property != None:
        print("child done")
        create_inferrence(parent, child_property, child)

    if parent_property != None:
        print("parent done")
        create_inferrence(child2, parent_property, parent2)
    
    if grandchild_property != None:
        print("grand child done")
        create_inferrence(grandparent, grandchild_property, grandchild)

    if grandparent_property != None:
        print("grand parent done")
        create_inferrence(grandchild2, grandparent_property, grandparent2)
        
    if husband_property != None:
        print("husband done")
        create_inferrence(marriage3, husband_property, marriage4)

    if wife_property != None:
        print("wife done")
        create_inferrence(marriage, wife_property, marriage2)

    if ex_husband_property!= None:
        print("ex husband done")
        create_inferrence(marriage7, ex_husband_property, marriage8)

    if ex_wife_property!= None:
        print("ex wife done")
        create_inferrence(marriage5, ex_wife_property, marriage6)

    if cousin_property!= None:
        print("cousin done")
        create_inferrence(cousin1, cousin_property, cousin2)

    if partner_property!= None:
        print("partner done")
        create_inferrence(couple, partner_property, couple2)

    if father_property!= None:
        print("father done")
        create_gender_based_inferrence(has_child_relation, father_property, fathers)

    if mother_property!= None:
        print("mother done")
        create_gender_based_inferrence(has_child_relation, mother_property, mothers)

    if son_property!= None:
        print("son done")
        create_gender_based_inferrence(has_parent_relation, son_property, sons)

    if daughter_property!= None:
        print("daughter done")
        create_gender_based_inferrence(has_parent_relation, daughter_property, daughters)

    if grandson_property!= None:
        print("grandson done")
        create_gender_based_inferrence(has_grandchild_relation, grandson_property, grandsons)

    if granddaugther_property!= None:
        print("granddaughter done")
        create_gender_based_inferrence(has_grandchild_relation, granddaugther_property, granddaughters)

    if grandfather_property!= None:
        print("grandfather done")
        create_gender_based_inferrence(has_grandparent_relation, grandfather_property, grandfathers)

    if grandmother_property!= None:
        print("grandmother done")
        create_gender_based_inferrence(has_grandparent_relation, grandmother_property, grandmothers)

    if brother_property!= None:
        print("brother done")
        create_gender_based_inferrence(has_sibling_relation, brother_property, brothers)

    if sister_property != None:
        print("sister done")
        create_gender_based_inferrence(has_sibling_relation, sister_property, sisters)

    if twin_property != None:
        print("twin done")
        create_gender_based_inferrence(has_twin_relation, twin_property, twin_sisters)
        create_gender_based_inferrence(has_twin_relation, twin_property, twin_brothers)
        

    if sibling_property != None:
        print("sibling done")
        create_inferrence(sibling1, sibling_property, sibling2)  
        
    else:
        print("None")
        
        


In [17]:
inferences()

child done
parent done
cousin done
father done
mother done
son done
daughter done
brother done
sister done
twin done
sibling done


## Show dataframes of the inferences that are made. 

In [18]:
df2 = df2.drop_duplicates().sort_values(by=['Property'])
df2.reset_index(drop=True)

,Subject,Property,Object
0,Monica,has_Child,Rachel
1,Monica,has_Child,Phoebe
2,Joey,has_Child,Rachel
3,Phoebe,has_Child,Rachel
4,Kathy,has_Child,Chandler
5,Rachel,has_Child,Paul
6,Earl,has_Child,Phoebe
7,Monica,has_Child,Mr._Geller
8,Terry,has_Child,Rachel
9,Rachel,has_Child,Janice


In [19]:
df5 = df5.drop_duplicates().sort_values(by=['Property'])
df5.reset_index(drop=True)

,Subject,Property,Object
0,Malcom,has_Brother,Phoebe
1,Ross,has_Brother,Ross
2,Ross,has_Brother,Joey
3,Joey,has_Brother,Monica
4,Janine,has_Brother,Ross
5,Phoebe,has_Brother,Ross
6,Jill,has_Brother,Rachel
7,Joey,has_Brother,Phoebe
8,Frank,has_Brother,Phoebe
9,Phoebe,has_Brother,Rachel


In [20]:
serialize()

@prefix kinship1: <http://www.semanticweb.org/combots#> .
@prefix ns1: <http://www.semanticweb.org/combots#Mrs._Verhoeven'> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

kinship1:Big_Nosed_Rachel kinship1:has_Parents kinship1:Fat_Monica .

kinship1:Dr._Green kinship1:has_Dauter kinship1:Ross .

kinship1:Joanne kinship1:has_Dad kinship1:Kiki .

kinship1:Joshua kinship1:has_Parents kinship1:Rachel .

kinship1:Pizza_Guy kinship1:has_Dad kinship1:Monica .

kinship1:Tom kinship1:has_Mom kinship1:Rachel .

kinship1:has_Child a kinship1:IrreflexiveProperty .

kinship1:has_Sibling a kinship1:SymmetricProperty .

kinship1:has_daughter a kinship1:IrreflexiveProperty .

kinship1:has_parent a kinship1:IrreflexiveProperty .

kinship1:Angela kinship1:has_Brother kinship1:Monica ;
    kinship1:has_Sibling kinship1:Monica ;
    kinship1:has_Sister kinship1:Monica ;
    rdfs:domain kinship1:man .

kinship1:Danny kinship1:has_Sister kinship1:R

# Add sibling relations

In [21]:
sibling_infer_relation_list = []
sibling_infer_relation_list_show = []
name_list_sibling = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_Sibling" or relation(p) == "has_Sister" or relation(p) == "has_Brother"  or relation(p) == "has_twin":
#         print(relation(s), relation(p), relation(o))
        sibling_infer_relation_list_show.append((relation(s), relation(o)))
        sibling_infer_relation_list.append((s, o))
        name_list_sibling.append(relation(s))
        name_list_sibling.append(relation(o))

In [22]:
sibling_infer_relation_list_show

[('Rachel', 'Phoebe'),
 ('Phoebe', 'Malcom'),
 ('Joey', 'Phoebe'),
 ('Joey', 'Chandler'),
 ('Joey', 'Dina'),
 ('Chandler', 'Phoebe'),
 ('Rachel', 'Joey'),
 ('Rachel', 'Joey'),
 ('Ross', 'All'),
 ('Ross', 'Ross'),
 ('All', 'Ross'),
 ('Dr._Leedbetter', 'Ross'),
 ('Joey', 'Rachel'),
 ('Rachel', 'All'),
 ('All', 'Rachel'),
 ('Chandler', 'Ross'),
 ('Dina', 'Joey'),
 ('Phoebe', 'Monica'),
 ('Chandler', 'Ross'),
 ('Ross', 'Monica'),
 ('Phoebe', 'Ross'),
 ('MNCA', 'ROSS'),
 ('Monica', 'Phoebe'),
 ('Monica', 'Ross'),
 ('Rachel', 'Monica'),
 ('Joey', 'Dina'),
 ('Ross', 'Ross'),
 ('Rachel', 'Phoebe'),
 ('Tag', 'Rachel'),
 ('Emily', 'Ross'),
 ('ROSS', 'MNCA'),
 ('Rachel', 'Jill'),
 ('Eric', 'Chandler'),
 ('Joey', 'Rachel'),
 ('Angela', 'Monica'),
 ('Ross', 'Phoebe'),
 ('Phoebe', 'Eric'),
 ('Phoebe', 'Ursula'),
 ('Ross', 'Janine'),
 ('All', 'Ross'),
 ('Monica', 'Joey'),
 ('Ross', 'Monica'),
 ('Joey', 'Chandler'),
 ('Joey', 'Phoebe'),
 ('All', 'Rachel'),
 ('Joey', 'Phoebe'),
 ('Ross', 'Ross'),
 ('Ra

In [23]:
name_data = pd.DataFrame(name_list_sibling, columns = ["Names Of All Siblings"])
final_name_list = name_data.drop_duplicates().reset_index(drop=True)
final_name_list

,Names Of All Siblings
0,Rachel
1,Phoebe
2,Malcom
3,Joey
4,Chandler
5,Dina
6,Ross
7,All
8,Dr._Leedbetter
9,Monica


### Put all the individuals that are siblings of each other in a set together and put those sets in a list. 

In [24]:
synom = sibling_infer_relation_list
show = sibling_infer_relation_list_show
word_classes = make_word_classes(synom)
# print(word_classes)
print(make_word_classes(show))

[{'Joey', 'Rachel', 'Dr._Leedbetter', 'Ross', 'Paul', 'Tag', 'Monica', 'All', 'Emily', 'Nurse', 'Richard', 'Malcom', 'Frank', 'Danny', 'Dina', 'Pete', 'Frannie', 'Tina', 'Ursula', 'Phoebe', 'Janine', 'Angela', 'Chandler', 'Eric', 'Jill'}, {'All', 'Dr._Leedbetter', 'Ross'}, {'MNCA', 'ROSS'}]


In [25]:
parent_infer_relation_list = []
parent_infer_relation_list_show = []
name_list_children = []
name_list_show = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        parent_infer_relation_list_show.append((relation(s), relation(o)))
        parent_infer_relation_list.append((s, o))
        name_list_show.append(relation(s))
        name_list_children.append(s)
# parent_infer_relation_list_show
# name_list_children

In [26]:
name_data = pd.DataFrame(name_list_show, columns = ["Names Of All children"])
final_name_list = name_data.drop_duplicates().reset_index(drop=True)
final_name_list

,Names Of All children
0,Ross
1,Rachel
2,Emily
3,Joey
4,Phoebe
5,Mr._Geller
6,Dr._Harad
7,Mrs._Geller
8,Chandler
9,Ben


### Create a list of the siblings and infer the sibling relations between them. 

In [27]:
new_list = []
end_list = []

for x in word_classes:
    for p in x:
        new_list.append(p)
        if p not in name_list_children:
            new_list.remove(p)
    
for x in word_classes:
    for people in new_list:
        for p in x:
            if p == people:
                end_list.append(x)     
end_list

[{rdflib.term.URIRef('http://www.semanticweb.org/combots#All'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Angela'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Chandler'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Danny'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Dina'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Dr._Leedbetter'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Emily'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Eric'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Frank'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Frannie'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Janine'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Jill'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Joey'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Malcom'),
  rdflib.term.URIRef('http://www.semanticweb.org/combots#Monica')

In [28]:
inferences()

child done
parent done
cousin done
father done
mother done
son done
daughter done
brother done
sister done
twin done
sibling done


In [29]:
df2 = df2.drop_duplicates().sort_values(by=['Property'])
df2.reset_index(drop=True)

,Subject,Property,Object
0,Monica,has_Child,Rachel
1,Rachel,has_Child,Rachel
2,Ross,has_Child,Rachel
3,Rachel,has_Child,Monica
4,Chandler,has_Child,Monica
5,Monica,has_Child,Chandler
6,Ross,has_Child,Elizabeth
7,Rachel,has_Child,Ross
8,Joey,has_Child,Mrs._Tribbiani
9,Mr._Geller,has_Child,Ross


In [30]:
df5 = df5.drop_duplicates().sort_values(by=['Property'])
df5.reset_index(drop=True)

,Subject,Property,Object
0,Malcom,has_Brother,Phoebe
1,Ross,has_Brother,Rachel
2,Phoebe,has_Brother,Frank
3,Angela,has_Brother,Monica
4,Eric,has_Brother,Phoebe
5,Ursula,has_Brother,Phoebe
6,Frannie,has_Brother,Monica
7,Rachel,has_Brother,Joey
8,Paul,has_Brother,Monica
9,Pete,has_Brother,Phoebe


# Show final ontology

In [31]:
serialize()

@prefix kinship1: <http://www.semanticweb.org/combots#> .
@prefix ns1: <http://www.semanticweb.org/combots#Mrs._Verhoeven'> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

kinship1:Big_Nosed_Rachel kinship1:has_Parents kinship1:Fat_Monica .

kinship1:Dr._Green kinship1:has_Dauter kinship1:Ross .

kinship1:Joanne kinship1:has_Dad kinship1:Kiki .

kinship1:Joshua kinship1:has_Parents kinship1:Rachel .

kinship1:Pizza_Guy kinship1:has_Dad kinship1:Monica .

kinship1:Tom kinship1:has_Mom kinship1:Rachel .

kinship1:has_Child a kinship1:IrreflexiveProperty .

kinship1:has_Sibling a kinship1:SymmetricProperty .

kinship1:has_daughter a kinship1:IrreflexiveProperty .

kinship1:has_parent a kinship1:IrreflexiveProperty .

kinship1:Angela kinship1:has_Brother kinship1:Monica ;
    kinship1:has_Sibling kinship1:Monica ;
    kinship1:has_Sister kinship1:Monica ;
    rdfs:domain kinship1:man .

kinship1:Danny kinship1:has_Sister kinship1:R

In [32]:
save('new_rawdata2_ontology.ttl')

# Show the result of the counter of the newly added relations to the ontology


In [33]:
new_relations_counter

718